In [1]:
!pip install ifcopenshell rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.7 MB/s eta 0:00:00


In [13]:
import ifcopenshell
import ifcopenshell.geom
import shapely.geometry
from rdflib import Graph, Namespace, URIRef, Literal, RDF

IFC_PATH = "/content/Building_Architecture.ifc"

ARC = Namespace("http://example.org/arc#")
IFC = Namespace("https://standards.buildingsmart.org/IFC/DEV/IFC4/ADD2_TC1/OWL")
GEO = Namespace("http://www.opengis.net/ont/geosparql#")

g = Graph()
g.bind("str", ARC)
g.bind("ifc", IFC)
g.bind("geo", GEO)

model = ifcopenshell.open(IFC_PATH)
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_WORLD_COORDS, True)

for element in model.by_type("IfcProduct"):
    try:
        shape = ifcopenshell.geom.create_shape(settings, element)
        verts = shape.geometry.verts  # flat list: x1, y1, z1, x2, y2, z2, ...
        coords = [(verts[i], verts[i + 1], verts[i + 2]) for i in range(0, len(verts), 3)]

        # Use 2D projection for WKT (simplified for RDF)
        points_2d = [(x, y) for x, y, z in coords]
        linestring = shapely.geometry.LineString(points_2d)
        wkt = linestring.wkt

        element_type = element.is_a()
        element_uri = URIRef(ARC + element.GlobalId.replace('$', '_'))
        geom_uri = URIRef(element_uri + "_geom")

        g.add((element_uri, RDF.type, IFC[element_type]))
        g.add((element_uri, GEO.hasGeometry, geom_uri))
        g.add((geom_uri, GEO.asWKT, Literal(wkt, datatype=GEO.wktLiteral)))

    except Exception as e:
        print(f"Skipped element {element.GlobalId} ({element.is_a()}): {e}")

# Save RDF to Turtle file
g.serialize("Building-Arc_Geo.ttl", format="turtle")


Skipped element 1wADrO19H3w980h1wUyXLk (IfcBuildingElementProxy): Representation is NULL
Skipped element 0bo7_K6az7AA$4RxkSNVNM (IfcBuildingElementProxy): Representation is NULL
Skipped element 3dkFAzOGrAIuOzY_RdrdVv (IfcChimney): Representation is NULL
Skipped element 2iPwJwpPDCSgMheXwk9cBT (IfcRoof): Representation is NULL
Skipped element 1Ano2ZUxnEIvVQ_beukl8b (IfcBuildingStorey): Representation is NULL
Skipped element 0c$N1CTon2BB2Sp89385G8 (IfcBuilding): Representation is NULL
Skipped element 23sFQGRy90RxVbRHD9iSE2 (IfcSite): Representation is NULL
Skipped element 1Pbuu0tu59NfhrTsztVBK1 (IfcSite): Representation is NULL
RDF export complete: infra_site_all_geometry.ttl
